In [22]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\FFF\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [1]:
import sys
from facenet_pytorch import InceptionResnetV1
sys.path.append('./License_Plate_Detection_Pytorch/LPRNet')
sys.path.append('./License_Plate_Detection_Pytorch/MTCNN')
import torch
from PIL import Image
import cv2
import cvzone
import matplotlib.pyplot as plt
import torchvision
import argparse
import os
import numpy as np
from collections import Counter
from datetime import datetime
from MTCNN import *
from LPRNet_Test import *
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import time
import easyocr
harcascade = "haarcascade_russian_plate_number.xml"

In [2]:
import cv2
harcascade = "haarcascade_russian_plate_number.xml"
plate_detector = cv2.CascadeClassifier(harcascade)
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, threshold = cv2.threshold(gray_blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    morph_opening = cv2.morphologyEx(threshold, cv2.MORPH_OPEN, kernel)
    contours, _ = cv2.findContours(morph_opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 500:
            cv2.drawContours(frame, [contour], 0, (0, 255, 0), 2)
            x, y, w, h = cv2.boundingRect(contour)
            cv2.putText(frame, "Contour", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, "Edge", (x + w - 50, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.imshow("Edges and Contours", frame)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [30]:
cap = cv2.VideoCapture(0)

cap.set(3, 640) # width
cap.set(4, 480) #height

min_area = 500
count = 0
reader = easyocr.Reader(['en'])

def calculate_blur(image, threshold=500):
    """
    Calculate the blur level of an image and check if it meets the acceptance threshold.
    
    Args:
        image (numpy.ndarray): The input image.
        threshold (float): The minimum variance of Laplacian to accept the image as "sharp".
    
    Returns:
        blur_level (float): The calculated variance of the Laplacian.
        is_acceptable (bool): True if the image is sharp enough, False otherwise.
    """
    # Convert image to grayscale for calculation
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Calculate the Laplacian
    laplacian = cv2.Laplacian(gray_image, cv2.CV_64F)
    
    # Calculate the variance (a measure of sharpness)
    blur_level = laplacian.var()
    
    # Check if the blur level meets the threshold
    is_acceptable = blur_level > threshold
    
    return blur_level, is_acceptable

while True:
    success, img = cap.read()

    plate_cascade = cv2.CascadeClassifier(harcascade)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    plates = plate_cascade.detectMultiScale(img_gray, 1.1, 4)

    for (x,y,w,h) in plates:
        area = w * h

        if area > min_area:
            cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 2)
            plate_img = img[y: y+h, x:x+w]

            _, acc = calculate_blur(plate_img)
            #print(_)
            if acc:
                #print(_)
                result = reader.readtext(img[y: y+h, x:x+w])
                if(len(result) > 0 and result[0][2] >.94 ):
                    cv2.putText(img, str(result[0][1]) + str(result[0][2]), (x,y-5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 0, 255), 2)

                    print(str(result[0][1]) + " score:" + str(result[0][2]))
                img_roi = img[y: y+h, x:x+w]
                cv2.imshow("ROI", img_roi)


    
    cv2.imshow("Result", img)
    if cv2.waitKey(1) == ord('q'):
        break
    if cv2.waitKey(1) & 0xFF == ord('s'):
        cv2.imwrite("plates/scaned_img_" + str(count) + ".jpg", img_roi)
        cv2.rectangle(img, (0,200), (640,300), (0,255,0), cv2.FILLED)
        cv2.putText(img, "Plate Saved", (150, 265), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0, 0, 255), 2)
        cv2.imshow("Results",img)
        cv2.waitKey(500)
        count += 1

cap.release()
cv2.destroyAllWindows()

QRNZE33 score:0.9841159525620041
BR score:0.972437041308365
BR score:0.9662023583554097
BR score:0.9700844375614486
QRMZE score:0.9662026206865605
QRNZE score:0.9663727869921924
QRMZE33 score:0.9776517165524792
QRMZE33 score:0.9438834897844725
BR score:0.9531953214294024
BR score:0.9583258550228483
BR score:0.9525173945730416
BR score:0.9492163827354839
BR score:0.9737054261814505
BR score:0.9841737862660972
BR score:0.9423103812862955
BR score:0.9625841570390731
BR score:0.9530812848651387
QRMZE score:0.9897272592451043
BR score:0.9728863728034635
QRMZE3 score:0.9759694212078242
BR score:0.9627923905414623
BR score:0.9625855741160012


In [23]:
import sys
from facenet_pytorch import InceptionResnetV1
sys.path.append('./License_Plate_Detection_Pytorch/LPRNet')
sys.path.append('./License_Plate_Detection_Pytorch/MTCNN')
import torch
from PIL import Image
import cv2
import cvzone
import matplotlib.pyplot as plt
import torchvision
import argparse
from glob import glob
import os
import random
import numpy as np
from collections import Counter
from datetime import datetime
#from MTCNN import *
#from LPRNet_Test import *
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import time
import easyocr
harcascade = "haarcascade_russian_plate_number.xml"
img = cv2.imread('car_1.png')

reader = easyocr.Reader(['pt'], recog_network='latin_g2', gpu=True)



def calculate_blur(image, threshold=500):
    """
    Calculate the blur level of an image and check if it meets the acceptance threshold.
    
    Args:
        image (numpy.ndarray): The input image.
        threshold (float): The minimum variance of Laplacian to accept the image as "sharp".
    
    Returns:
        blur_level (float): The calculated variance of the Laplacian.
        is_acceptable (bool): True if the image is sharp enough, False otherwise.
    """
    # Convert image to grayscale for calculation
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Calculate the Laplacian
    laplacian = cv2.Laplacian(gray_image, cv2.CV_64F)
    
    # Calculate the variance (a measure of sharpness)
    blur_level = laplacian.var()
    
    # Check if the blur level meets the threshold
    is_acceptable = blur_level > threshold
    
    return blur_level, is_acceptable


plate_detector = cv2.CascadeClassifier(cv2.data.haarcascades + harcascade)
plate_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + harcascade)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

if img is None:
    print("Error: Image not found!")
else:
    cv2.imshow("IMG", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

yolo_plate_model = YOLO("yolov8n.pt")  
#plates = plate_cascade.detectMultiScale(img_gray, 1.001, 4)
#print(plates)
plates = yolo_plate_model(img)
if len(plates) > 0:
    for (x,y,w,h) in plates:
        area = w * h
    
        if area > 100 and area < 50000:
            cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 2)
            plate_img = img[y: y+h, x:x+w]
    
            _, acc = calculate_blur(plate_img)
            #print(_)
            if acc:
                #print(_)
                result = reader.readtext(img[y: y+h, x:x+w])
                if(len(result) > 0 and result[0][2] >.2):
                    cv2.putText(img, str(result[0][1]) + str(result[0][2]), (x,y-5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 0, 255), 2)
    
                    print(str(result[0][1]) + " score:" + str(result[0][2]))
                img_roi = img[y: y+h, x:x+w]
                #cv2.imshow("ROI", img_roi)
                #cv2.waitKey(0)
                #cv2.destroyAllWindows()
    
    
    cv2.imshow("Result", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    


#cap.release()
#cv2.destroyAllWindows()

100%|█████████████████████████████████████████████████████████████████████| 6.25M/6.25M [00:00<00:00, 11.8MB/s]



WARNING  NMS time limit 2.050s exceeded
0: 384x640 1 car, 1 truck, 80.8ms
Speed: 60.5ms preprocess, 80.8ms inference, 7307.0ms postprocess per image at shape (1, 3, 384, 640)


ValueError: not enough values to unpack (expected 4, got 2)

In [41]:
import sys
from facenet_pytorch import InceptionResnetV1
sys.path.append('./License_Plate_Detection_Pytorch/LPRNet')
sys.path.append('./License_Plate_Detection_Pytorch/MTCNN')
import torch
from PIL import Image
import cv2
import cvzone
import matplotlib.pyplot as plt
import torchvision
import argparse
from glob import glob
import os
import random
import numpy as np
from collections import Counter
from datetime import datetime
#from MTCNN import *
#from LPRNet_Test import *
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import time
import easyocr
harcascade = "haarcascade_russian_plate_number.xml"
img = cv2.imread('2.png')

reader = easyocr.Reader(['pt'], recog_network='latin_g2', gpu=True)



def calculate_blur(image, threshold=500):
    """
    Calculate the blur level of an image and check if it meets the acceptance threshold.
    
    Args:
        image (numpy.ndarray): The input image.
        threshold (float): The minimum variance of Laplacian to accept the image as "sharp".
    
    Returns:
        blur_level (float): The calculated variance of the Laplacian.
        is_acceptable (bool): True if the image is sharp enough, False otherwise.
    """
    # Convert image to grayscale for calculation
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Calculate the Laplacian
    laplacian = cv2.Laplacian(gray_image, cv2.CV_64F)
    
    # Calculate the variance (a measure of sharpness)
    blur_level = laplacian.var()
    
    # Check if the blur level meets the threshold
    is_acceptable = blur_level > threshold
    
    return blur_level, is_acceptable


plate_detector = cv2.CascadeClassifier(cv2.data.haarcascades + harcascade)
plate_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + harcascade)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

if img is None:
    print("Error: Image not found!")
else:
    cv2.imshow("IMG", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

yolo_plate_model = YOLO("best_plate.pt")  
#plates = plate_cascade.detectMultiScale(img_gray, 1.001, 4)
#print(plates)
results = yolo_plate_model(img)
if len(plates) > 0:
    for r in results:
        for box in r.boxes:
            x, y, w, h = map(int, box.xyxy[0])  # Bounding box coordinates
            cv2.rectangle(img, (x-10, y-10), (w+10, h+10), (0, 255, 0), 2)
            plate_img = img[y: y+h, x:x+w]
            _, acc = calculate_blur(plate_img)
            #print(_)
            if acc:
                #print(_)
                result = reader.readtext(img[max(y-20, 0): y+h+20, max(x-20,0):x+w+20])
                if(len(result) > 0 and result[0][2] >.2):
                    cv2.putText(img, str(result[0][1]) + str(result[0][2]), (x,y-5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 0, 255), 2)
                    print(result)
                    print(str(result[0][1]) + " score:" + str(result[0][2]))
                img_roi = img[y: y+h, x:x+w]
                #cv2.imshow("ROI", img_roi)
                #cv2.waitKey(0)
                #cv2.destroyAllWindows()
    
    
    cv2.imshow("Result", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    


#cap.release()
#cv2.destroyAllWindows()


0: 512x640 2 Platess, 51.9ms
Speed: 5.0ms preprocess, 51.9ms inference, 3.0ms postprocess per image at shape (1, 3, 512, 640)
[([[31, 17], [175, 17], [175, 57], [31, 57]], '[RSGK', 0.2538915877464017)]
[RSGK score:0.2538915877464017


In [53]:
import sys
from facenet_pytorch import InceptionResnetV1
sys.path.append('./License_Plate_Detection_Pytorch/LPRNet')
sys.path.append('./License_Plate_Detection_Pytorch/MTCNN')
import torch
from PIL import Image
import cv2
import cvzone
import matplotlib.pyplot as plt
import torchvision
import argparse
from glob import glob
import os
import random
import numpy as np
from collections import Counter
from datetime import datetime
#from MTCNN import *
#from LPRNet_Test import *
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import time
import easyocr
harcascade = "haarcascade_russian_plate_number.xml"
img = cv2.imread('2.png')

reader = easyocr.Reader(
    lang_list=['en', 'fr'],      # English & French
    gpu=True,                    # Use GPU if available
#    model_storage_directory='./models',  # Custom model storage
    recog_network='english_g2',  # More accurate English model
    download_enabled=True        # Allow model download if needed
)



def calculate_blur(image, threshold=500):
    """
    Calculate the blur level of an image and check if it meets the acceptance threshold.
    
    Args:
        image (numpy.ndarray): The input image.
        threshold (float): The minimum variance of Laplacian to accept the image as "sharp".
    
    Returns:
        blur_level (float): The calculated variance of the Laplacian.
        is_acceptable (bool): True if the image is sharp enough, False otherwise.
    """
    # Convert image to grayscale for calculation
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Calculate the Laplacian
    laplacian = cv2.Laplacian(gray_image, cv2.CV_64F)
    
    # Calculate the variance (a measure of sharpness)
    blur_level = laplacian.var()
    
    # Check if the blur level meets the threshold
    is_acceptable = blur_level > threshold
    
    return blur_level, is_acceptable


plate_detector = cv2.CascadeClassifier(cv2.data.haarcascades + harcascade)
plate_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + harcascade)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

if img is None:
    print("Error: Image not found!")
else:
    cv2.imshow("IMG", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

yolo_plate_model = YOLO("best_plate.pt")  
#plates = plate_cascade.detectMultiScale(img_gray, 1.001, 4)
#print(plates)
results = yolo_plate_model(img)
if len(plates) > 0:
    for r in results:
        for box in r.boxes:
            x, y, w, h = map(int, box.xyxy[0])  # Bounding box coordinates
            cv2.rectangle(img, (x-10, y-10), (w+10, h+10), (0, 255, 0), 2)
            plate_img = img[y: y+h, x:x+w]
            _, acc = calculate_blur(plate_img)
            #print(_)
            if acc:
                #print(_)
                result = reader.readtext(img[max(y, 0): y+h, max(x,0):x+w])
                if(len(result) > 0 and result[0][2] >.2):
                    cv2.putText(img, str(result[0][1]) + " "+str(float(result[0][2]*100, 2)) + "%" , (x,y-5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 0, 255), 2)
                    print(result)
                    print(str(result[0][1]) + " score:" + str(result[0][2]))
                img_roi = img[y: y+h, x:x+w]
                #cv2.imshow("ROI", img_roi)
                #cv2.waitKey(0)
                #cv2.destroyAllWindows()
    
    
    cv2.imshow("Result", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    


#cap.release()
#cv2.destroyAllWindows()


0: 512x640 2 Platess, 14.0ms
Speed: 2.0ms preprocess, 14.0ms inference, 3.0ms postprocess per image at shape (1, 3, 512, 640)


TypeError: float expected at most 1 argument, got 2

In [8]:
harcascade = "haarcascade_russian_plate_number.xml"
plate_detector = cv2.CascadeClassifier(harcascade)


frame = cv2.imread('car_1.png')
if frame is None:
    print("Error: Image not found!")
else:
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, threshold = cv2.threshold(gray_blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    morph_opening = cv2.morphologyEx(threshold, cv2.MORPH_OPEN, kernel)
    contours, _ = cv2.findContours(morph_opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 500:
            cv2.drawContours(frame, [contour], 0, (0, 255, 0), 2)
            x, y, w, h = cv2.boundingRect(contour)
            cv2.putText(frame, "Contour", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, "Edge", (x + w - 50, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.imshow("Edges and Contours", frame)
    
cv2.waitKey(0)
cv2.destroyAllWindows()
